In [1]:
pip install azure-mgmt-resource azure-identity azure-ai-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, datetime
from azure.identity import DeviceCodeCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Environment

cred = DeviceCodeCredential()
ml_client = MLClient(
    credential=cred,
    subscription_id="PREGUNTAR_A_HIBRAN",
    resource_group_name="azure-for-students",
    workspace_name="rar-solutions-ml-workspace"
)

# 1) Crea el Endpoint
endpoint_name = f"hotel-forecast-{datetime.datetime.now():%Y%m%d%H%M%S}"
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="LightGBM hotel occupancy forecast",
    auth_mode="key",
)
ml_client.online_endpoints.begin_create_or_update(endpoint).wait()
print(f"Endpoint creado: {endpoint_name}")

# 2) Crea el Environment usando la imagen de Microsoft
env = Environment(
    name="lgb-inference-env",
    description="LightGBM + scikit-learn env basado en imagen curated",
    image="mcr.microsoft.com/azureml/curated/lightgbm-3.3:61",
    os_type="linux",
)
ml_client.environments.create_or_update(env)
print("Entorno “lgb-inference-env” creado o actualizado correctamente")

# 3) Crea el Deployment “blue”
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=ml_client.models.get(name="hotel-lightgbm-occupancy", version="3"),
    environment=env,           
    code_path="/content/",      
    scoring_script="score.py",
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(deployment).wait()
print("Deployment “blue” creado")

# 4) Enruta 100 % del tráfico a “blue”
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).wait()
print("Deployment listo y recibiendo tráfico")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B9WB9ZDR4 to authenticate.
✅ Endpoint creado: hotel-forecast-20250604072405
✅ Entorno “lgb-inference-env” creado o actualizado correctamente


Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint hotel-forecast-20250604072405 exists


..................................................................................✅ Deployment “blue” creado
🚀 Deployment listo y recibiendo tráfico
